In [1]:
import os, shutil
import numpy as np
import tensorflow as tf
base_dir = 'tiles'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

/home/miguelmmanso/anaconda3/envs/pruebas/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/miguelmmanso/anaconda3/envs/pruebas/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/miguelmmanso/anaconda3/envs/pruebas/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/miguelmmanso/anacon

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [3]:
import keras
from keras import layers
from keras import models
from keras.applications import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 256, 256
conv_base= VGG16(weights=None, include_top=False, input_shape=(img_width,img_height,3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=4096,activation="relu")) #si no entrena bien: init = 'he_normal'    #he_uniform
model.add(layers.Dense(units=4096,activation="relu"))
model.add(layers.Dense(units=1, activation="sigmoid"))

model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              33558528  
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4097      
Total params: 65,058,625
Trainable params: 65,058,625
Non-trainable params: 0
________________________________________________________________

In [4]:
from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-4), metrics=['acc']) #si es demasiado lento, Adam(lr=0.001)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1. /255)
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=25, 
                                   width_shift_range=0.1, height_shift_range=0.1, 
                                   zoom_range=0.1, horizontal_flip=True, vertical_flip=True, 
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(256, 256),
                                                    batch_size=50, class_mode='binary')
validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(256, 256), 
                                                              batch_size=50,class_mode='binary')

Found 484048 images belonging to 2 classes.
Found 26893 images belonging to 2 classes.


In [6]:
train_steps_per_epoch = np.math.ceil(train_generator.samples / train_generator.batch_size)
validation_steps_per_epoch = np.math.ceil(validation_generator.samples / validation_generator.batch_size)

#teniendo en cuenta el desbalanceo de clases
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(train_generator.classes), train_generator.classes)
class_weights

/home/miguelmmanso/anaconda3/envs/pruebas/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


array([1.03952376, 0.96337163])

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch= train_steps_per_epoch, 
                              epochs=50, validation_data=validation_generator,
                              validation_steps=validation_steps_per_epoch, class_weight=class_weights) #callbacks = [tensorboard]



Epoch 1/50
9681/9681 [==============================] - 4988s 515ms/step - loss: 0.5138 - acc: 0.7290 - val_loss: 0.4430 - val_acc: 0.7851
Epoch 2/50
9681/9681 [==============================] - 4960s 512ms/step - loss: 0.4188 - acc: 0.8000 - val_loss: 0.3853 - val_acc: 0.8152
Epoch 3/50
9681/9681 [==============================] - 4958s 512ms/step - loss: 0.3933 - acc: 0.8117 - val_loss: 0.3899 - val_acc: 0.8208
Epoch 4/50
9681/9681 [==============================] - 4959s 512ms/step - loss: 0.3778 - acc: 0.8184 - val_loss: 0.3661 - val_acc: 0.8282
Epoch 5/50
9681/9681 [==============================] - 4959s 512ms/step - loss: 0.3674 - acc: 0.8223 - val_loss: 0.3650 - val_acc: 0.8269
Epoch 6/50
9681/9681 [==============================] - 4965s 513ms/step - loss: 0.3584 - acc: 0.8251 - val_loss: 0.3695 - val_acc: 0.8236
Epoch 7/50
9681/9681 [==============================] - 4968s 513ms/step - loss: 0.3522 - acc: 0.8267 - val_loss: 0.3729 - val_acc: 0.8257
Epoch 8/50
9681/9681 [====

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.figure(figsize=(20,10))
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) +1) 
plt.rcParams.update({'font.size':18})
plt.plot(epochs, acc, 'bo--', color='r', label='Training acc')
plt.plot(epochs, val_acc, 'bo--', color='b', label='Validation acc')
plt.legend()
plt.plot(epochs, loss, color= 'r', label='Training loss')
plt.plot(epochs, val_loss, color="b", label='Validation loss')
plt.legend()
plt.show()

In [ ]:
model.save("models1/vgg16_noweights-2.h5")

In [ ]:
#EARLY STOPPING
#from keras.callbacks import ModelCheckpoint, EarlyStopping
#checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1,
#                             save_best_only=True, save_weights_only=False, mode='auto', 
#                             period=5)
#early = EarlyStopping(monitor='val_acc', min_delta=0, patience=50, verbose=1, mode='auto')

#hist = model.fit_generator(steps_per_epoch=100,generator=traindata, 
#                           validation_data= testdata, validation_steps=10,epochs=100,
#                           callbacks=[checkpoint,early])

In [ ]:
#from keras.preprocessing import imageimg = image.load_img("image.jpeg",target_size=(224,224))
#img = np.asarray(img)
#plt.imshow(img)
#img = np.expand_dims(img, axis=0)from keras.models import load_model
#saved_model = load_model("vgg16_1.h5")output = saved_model.predict(img)
#if output[0][0] > output[0][1]:
#    print("Road")
#else:
#    print('No Road')